## Introduction:
You can use this ipython notebook as a template for the rest of the homework.

### 0. Basic Useful Setups:

In [1]:
### Basic setups
import sys
sys.path.append('./models/')
# Enable automatic reload of libraries
%load_ext autoreload
# All modules are reloaded before every comment
%autoreload 2
import keras

/opt/conda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 1. Read MNIST using Keras

In [2]:
from utils import load_mnist
(x_train, y_train), (x_test, y_test) = load_mnist()

('x_train shape:', (60000, 28, 28, 1))
(60000, 'train samples')
(10000, 'test samples')


In [9]:
from keras.layers import (
    Conv2D, BatchNormalization,
    MaxPooling2D, ZeroPadding2D, AveragePooling2D,
    add, Dense, Flatten
)
from keras.layers.advanced_activations import PReLU
def resnet(input_tensor):
    def name_builder(type, stage, block, name):
        return "{}{}{}_branch{}".format(type, stage, block, name)
    def identity_block(input_tensor, kernel_size, filters, stage, block):
        F1, F2, F3 = filters

        def name_fn(type, name):
            return name_builder(type, stage, block, name)

        x = Conv2D(F1, (1, 1), name=name_fn('res', '2a'))(input_tensor)
        x = BatchNormalization(name=name_fn('bn', '2a'))(x)
        x = PReLU()(x)

        x = Conv2D(F2, kernel_size, padding='same', name=name_fn('res', '2b'))(x)
        x = BatchNormalization(name=name_fn('bn', '2b'))(x)
        x = PReLU()(x)

        x = Conv2D(F3, (1, 1), name=name_fn('res', '2c'))(x)
        x = BatchNormalization(name=name_fn('bn', '2c'))(x)
        x = PReLU()(x)

        x = add([x, input_tensor])
        x = PReLU()(x)

        return x
    
    def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
        def name_fn(type, name):
            return name_builder(type, stage, block, name)

        F1, F2, F3 = filters

        x = Conv2D(F1, (1, 1), strides=strides, name=name_fn("res", "2a"))(input_tensor)
        x = BatchNormalization(name=name_fn("bn", "2a"))(x)
        x = PReLU()(x)

        x = Conv2D(F2, kernel_size, padding='same', name=name_fn("res", "2b"))(x)
        x = BatchNormalization(name=name_fn("bn", "2b"))(x)
        x = PReLU()(x)

        x = Conv2D(F3, (1, 1), name=name_fn("res", "2c"))(x)
        x = BatchNormalization(name=name_fn("bn", "2c"))(x)

        sc = Conv2D(F3, (1, 1), strides=strides, name=name_fn("res", "1"))(input_tensor)
        sc = BatchNormalization(name=name_fn("bn", "1"))(sc)

        x = add([x, sc])
        x = PReLU()(x)

        return x
    net = ZeroPadding2D((3, 3))(input_tensor)
    net = Conv2D(64, (7, 7), strides=(2, 2), name="conv1")(net)
    net = BatchNormalization(name="bn_conv1")(net)
    net = PReLU()(net)
    net = MaxPooling2D((3, 3), strides=(2, 2))(net)

    net = conv_block(net, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    net = identity_block(net, 3, [64, 64, 256], stage=2, block='b')
    net = identity_block(net, 3, [64, 64, 256], stage=2, block='c')

    net = conv_block(net, 3, [128, 128, 512], stage=3, block='a')
    net = identity_block(net, 3, [128, 128, 512], stage=3, block='b')
    net = identity_block(net, 3, [128, 128, 512], stage=3, block='c')
    net = identity_block(net, 3, [128, 128, 512], stage=3, block='d')

    net = conv_block(net, 3, [256, 256, 1024], stage=4, block='a')
    net = identity_block(net, 3, [256, 256, 1024], stage=4, block='b')
    net = identity_block(net, 3, [256, 256, 1024], stage=4, block='c')
    net = identity_block(net, 3, [256, 256, 1024], stage=4, block='d')
    net = identity_block(net, 3, [256, 256, 1024], stage=4, block='e')
    net = identity_block(net, 3, [256, 256, 1024], stage=4, block='f')
    net = AveragePooling2D((2, 2))(net)

    net = Flatten()(net)
    net = Dense(10, activation="softmax", name="softmax")(net)

    return net


In [17]:
from keras.models import Model
from keras.layers import Input
X = Input(shape=[28, 28, 1])
y = resnet(X)
model = Model(X, y, name="resnet")
model.compile("adam", "categorical_crossentropy", ["accuracy"])


### 2. Train and Evaluate LeNet Model

In [11]:
# train model
BATCH_SIZE = 128
MAX_EPOCH = 12

hist = model.fit(x_train, y_train, batch_size=BATCH_SIZE, nb_epoch=12, verbose=1)

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Epoch 1/12
60000/60000 [==============================] - 51s 852us/step - loss: 0.1863 - acc: 0.9461
Epoch 2/12
60000/60000 [==============================] - 44s 730us/step - loss: 0.0657 - acc: 0.9815
Epoch 3/12
60000/60000 [==============================] - 43s 719us/step - loss: 0.0358 - acc: 0.9892
Epoch 4/12
60000/60000 [==============================] - 45s 743us/step - loss: 0.0231 - acc: 0.9930
Epoch 5/12
60000/60000 [==============================] - 46s 767us/step - loss: 0.0220 - acc: 0.9931
Epoch 6/12
60000/60000 [==============================] - 46s 761us/step - loss: 0.0224 - acc: 0.9928
Epoch 7/12
60000/60000 [==============================] - 45s 753us/step - loss: 0.0185 - acc: 0.9940
Epoch 8/12
60000/60000 [==============================] - 45s 744us/step - loss: 0.0189 - acc: 0.9940
Epoch 9/12
60000/60000 [==============================] - 45s 755us/step - loss: 0.0147 - acc: 0.9952
Epoch 10/12
60000/60000 [==============================] - 46s 765us/step - loss: 

In [14]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.02614428688850312)
('Test accuracy:', 0.9929)


In [16]:
score

[0.02614428688850312, 0.9929]